In [ ]:
import glob
import os

import btrack
import json
from skimage import io
import matplotlib.pyplot as plt
from skimage.measure import regionprops
from skimage.io import imread
from btrack.dataio import HDF5FileHandler
from btrack.constants import BayesianUpdates
import numpy as np
from pathlib import Path
from skimage.io import imread
from skimage.io import imread
from skimage.util import montage

In [ ]:
# Define a function to make the files into a numpy array

def segmentation_arr(files):
    """Segmentation as numpy array."""
    
    stack = []
    for filename in files:
        img = imread(filename)
        stack.append(img)
    return np.stack(stack, axis=0)

In [ ]:
## User changes MASTERPATH only. This the globs through and selects all mask_avg folders in condition folders for process >

MASTERPATH = 'D://cellPLATO_test_data/TestSet/'

PATHTOCONFIG = 'initialization/btrack_config.json'


# CONFIG_FILE = datasets.cell_config()  # this is the default cell tracking config file

# PATHTOCONFIG ="D://GitHub_software_forallusers/BayesianTracker/BayesianTracker/models/cell_config_Michael_phase.json"
INPUT_FMT = 'cellpose2D' #cellpose #cellpose2D

if INPUT_FMT == 'usiigaci':       
    PATHCONDLIST = glob.glob(os.path.join(MASTERPATH, 'Condition*', '*_mask_avg'))
    
elif INPUT_FMT == 'cellpose':
    PATHCONDLIST = glob.glob(os.path.join(MASTERPATH, 'Condition*'))
    
elif INPUT_FMT == 'cellpose2D':
    PATHCONDLIST = glob.glob(os.path.join(MASTERPATH, 'Condition*', '*'))

elif INPUT_FMT == 'cellpose3D':
    PATHCONDLIST = glob.glob(os.path.join(MASTERPATH, 'Condition*', '*'))
        
elif INPUT_FMT == 'cellpose_skrt':
    PATHCONDLIST = glob.glob(os.path.join(MASTERPATH,  '*' ))
    
print(PATHCONDLIST)

# Display the PATHCONDLIST in a nice way
for i in PATHCONDLIST:
    print(i)


In [ ]:
if INPUT_FMT == 'usiigaci':
    scaling = (1., 1.)   
elif INPUT_FMT == 'cellpose3D':
    scaling = (3.45, 1., 1.)
    # scaling = (1., 1.)
elif INPUT_FMT == 'cellpose2D':    
    scaling = (1., 1.)

In [ ]:
FEATURES = [
    'area',
    'bbox_area',
    'eccentricity',
    'equivalent_diameter',
    'extent',
    'filled_area',
    'major_axis_length',
    'minor_axis_length',
    'orientation',
    'perimeter',
    'solidity',

]


In [ ]:
## Then extract the segmentation files iteratively, do tracking on them and save a h5 in place

for PATH in PATHCONDLIST:
    
    ww = os.path.dirname(PATH) #wheretosavethefile
    basenameyeah = os.path.basename(PATH)
    
    # files = glob.glob(os.path.join(PATH, '*.png')) #Use this line for Usiigaci
    # files = glob.glob(os.path.join(PATH, '*_cp_masks_T????.tif')) #
    files = glob.glob(os.path.join(PATH, '*_cp_masks.tif')) #Use this line and the part below for Cellpose

    for i in files:
        print(i)

    stack = []
    stack = segmentation_arr(files)
    obj_from_arr = []
    obj_from_arr = btrack.utils.segmentation_to_objects(stack, scale=scaling, properties=tuple(FEATURES))

    with btrack.BayesianTracker() as tracker:

        # configure the tracker using a config file
        tracker.configure_from_file(PATHTOCONFIG)
        tracker.max_search_radius = 50
        tracker.verbose = True

        tracker.tracking_updates = ["MOTION"] #update 2023
        tracker.features = FEATURES #update 2023 
        tracker.update_method = BayesianUpdates.EXACT #changed from EXACT
        # append the objects to be tracked
        tracker.append(obj_from_arr)
        # set the volume
        tracker.volume=((0, 2030), (0, 2030), (0, 100000.)) #changed for Batya
        # track them (in interactive mode)
        tracker.track_interactive(step_size=100)
        # generate hypotheses and run the global optimizer
        tracker.optimize()
        h5fileexport = os.path.join(ww, (basenameyeah + '_tracks.h5'))
        # export the tracks
        tracker.export(h5fileexport, obj_type='obj_type_1')
        # write the segmentation (you needed to add "a" rather than "w" here!)
        with HDF5FileHandler(h5fileexport, "a") as h:
            h.write_segmentation(stack)
            # make sure that we did this by checking that the data exists
            assert "segmentation" in h._hdf
        